# Word2Vec

Word2Vec es un modelo que se utiliza para aprender representaciones vectoriales de palabras. Estas representaciones pueden capturar muchas propiedades linguisticas de las palabras, como su significado semántico, gramatical y hasta contextual.

### 1. Extraer la Data

Detectar codificación

In [ ]:
!pip install chardet

In [48]:
import chardet

In [49]:
with open("./CaballeriaRoja_Babel.txt", "rb") as file:
    raw_data = file.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']
    print(f"Detected encoding: {encoding}")

Detected encoding: Windows-1252


In [ ]:
!pip install pypdf2 -q

In [50]:
import string
from gensim.models import Word2Vec
import PyPDF2

In [51]:
with open("./CaballeriaRoja_Babel.txt", "r", encoding="Windows-1252") as file:
    documento = file.read()

In [52]:
documento

'CABALLERÍA ROJA\n\nIsaac Babel\n(1894-c.1941 )\n\nLA HIJA\n\nEl comandante de la sexta división comunicó que al amanecer el nuevo día había que ocupar Novogrado-Volynsk. El estado mayor abandonó Krapivno, y nuestro convoy, con gran estruendo, quedó como retaguardia a lo largo de la carretera, de aquella indestructible carretera que va de Brest a Varsovia, mandada construir un día por Nicolás I con huesos de campesinos.\nFlorecen en torno, los campos de adormidera púrpura; el viento sur juguetea en los centenos amarillos; el tierno trigo sarraceno se recorta en el horizonte como el muro de un convento lejano. La apacible Volinia se extiende a nuestro lado; ante nosotros retrocede y se hunde en los bosques de abedules una niebla nacarina que escala luego las cuestas floridas, prendiéndose con sus tenues brazos a las ramas de lúpulo. El sol, de color naranja, rueda por el horizonte como una cabeza cortada; en las desgarraduras de las nubes se estremece una luz débil; sobre nuestras cabez

In [53]:
len(documento)

204238

### 2. Preprocesamiento de datos

El objetivo del preprocesamiento es convertir el documento en una lista de frases, y cada frase en una lista de palabras, eliminando signos de puntuación y convirtiendo todo a minúsculas.

In [54]:
# Dividir el documento en frases usando el punto como separador
oraciones = documento.split(".")
len(oraciones)

2616

In [55]:
oraciones[1]

'1941 )\n\nLA HIJA\n\nEl comandante de la sexta división comunicó que al amanecer el nuevo día había que ocupar Novogrado-Volynsk'

In [56]:
oraciones_limpias = []
for oracion in oraciones:
    # Eliminar puntuación y dividir por espacios
    tokens = oracion.translate(str.maketrans('', '', string.punctuation)).split()
    
    # Convertir a minúsculas y eliminar números
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Añadir solo si hay tokens
    if tokens:
        oraciones_limpias.append(tokens)

In [57]:
oraciones_limpias[1]

['la',
 'hija',
 'el',
 'comandante',
 'de',
 'la',
 'sexta',
 'división',
 'comunicó',
 'que',
 'al',
 'amanecer',
 'el',
 'nuevo',
 'día',
 'había',
 'que',
 'ocupar',
 'novogradovolynsk']

### 3. Entrenamiento del modelo Word2Vec

In [58]:
# Entrenar el modelo Word2Vec
model = Word2Vec(sentences=oraciones_limpias, vector_size=500, window=5, min_count=1, workers=8)

In [59]:
vector = model.wv['historia']

In [60]:
palabras_cercanas = model.wv.most_similar('historia', topn=10)
palabras_cercanas

[('en', 0.9983624219894409),
 ('que', 0.9983614087104797),
 ('un', 0.9983577728271484),
 ('es', 0.9983543753623962),
 ('le', 0.9983530044555664),
 ('su', 0.9983522295951843),
 ('de', 0.9983511567115784),
 ('nuestro', 0.9983500838279724),
 ('los', 0.9983497262001038),
 ('como', 0.9983471035957336)]

In [61]:
model.save("caballeria_roja.model")

In [62]:
modelo_cargado = Word2Vec.load("caballeria_roja.model")

In [63]:
palabras_cercanas = modelo_cargado.wv.most_similar("caballero", topn=10)
palabras_cercanas

[('algo', 0.9611254334449768),
 ('unos', 0.9607442617416382),
 ('tiene', 0.9606738686561584),
 ('hombres', 0.9606360197067261),
 ('tengo', 0.960549533367157),
 ('podía', 0.9605357646942139),
 ('ciudad', 0.9605245590209961),
 ('todos', 0.9604822993278503),
 ('poco', 0.9604784846305847),
 ('ni', 0.960443377494812)]

Guardar solo los embeddings

In [64]:
model.wv.save_word2vec_format("caballero_emb.txt", binary=False)
model.wv.save_word2vec_format("caballero_emb.bin", binary=True)

In [65]:
from gensim.models import KeyedVectors

In [67]:
embeddings_cargados = KeyedVectors.load_word2vec_format("caballero_emb.txt", binary=False)
# O si fue guardado en formato binario:
# embeddings_cargados = KeyedVectors.load_word2vec_format("caballero_emb.bin", binary=True)
embeddings_cargados

In [68]:
def analogias(v1,v2,v3):
    similitud = embeddings_cargados.most_similar(positive=[v1,v3], negative=[v2])
    print(f"{v1} es a {v2} como {similitud[0][0]} es a {v3}")

In [69]:
analogias("caballero","hombre","mujer")

caballero es a hombre como algo es a mujer


### Prueba modelo 100 años de soleadad

In [70]:
def extraer_texto_desde_pdf(ruta_archivo):
    with open(ruta_archivo, "rb") as archivo:
        lector = PyPDF2.PdfReader(archivo)
        texto = ""
        for pagina in range(len(lector.pages)):
            texto += lector.pages[pagina].extract_text()
    return texto

In [71]:
documento = extraer_texto_desde_pdf("./cien-anos-de-soledad-gabriel-garcia-marquez.pdf")
documento

"CIEN AÑOS DE \nSOLEDAD\nGabriel García Márquez\nInfoLibros.org\nS I N O P S I S   D E   C I E N   A Ñ O S   D E   S O L E D A D \n \nCien años de soledad es sin lugar a dudas la novela má s \ncélebre de la literatura hispanoamericana del siglo XX. Algunos \ncríticos han enmarcado a Cien años de soledad en lo que se \nconoce como el realismo mágico o “lo real maravilloso”, una \ncorriente literaria de los países latinoamericanos donde la \nrealidad co nvive  con lo sobrenatural, mientras los personajes \nde las obras experimentan el fenómeno sin ningún asombro.  \n \nCien años de soledad narra la historia de los Buendía a lo largo \nde 100 años, una familia que habitaba el pueblo de Macondo. \nTanto los nombres, c omo los miedos, obsesiones y fracasos \nsuelen repetirse una y otra vez dentro de la trama, dejando al \nlector con la sensación de que los personajes están atrapados \npor la fuerza de la herencia familiar.   \n \nCien años de soledad por Gabriel García Márquez  en \nInfoLib

In [72]:
len(documento)

832032

In [74]:
# Dividir el documento en frases usando la coma como separador
oraciones = documento.split(",")
len(oraciones)

8862

In [75]:
oraciones[0]

'CIEN AÑOS DE \nSOLEDAD\nGabriel García Márquez\nInfoLibros.org\nS I N O P S I S   D E   C I E N   A Ñ O S   D E   S O L E D A D \n \nCien años de soledad es sin lugar a dudas la novela má s \ncélebre de la literatura hispanoamericana del siglo XX. Algunos \ncríticos han enmarcado a Cien años de soledad en lo que se \nconoce como el realismo mágico o “lo real maravilloso”'

In [76]:
oraciones_limpias = []
for oracion in oraciones:
    # Eliminar puntuación y dividir por espacios
    tokens = oracion.translate(str.maketrans('', '', string.punctuation)).split()
    
    # Convertir a minúsculas y eliminar números
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Añadir solo si hay tokens
    if tokens:
        oraciones_limpias.append(tokens)

In [77]:
oraciones_limpias[1]

['una',
 'corriente',
 'literaria',
 'de',
 'los',
 'países',
 'latinoamericanos',
 'donde',
 'la',
 'realidad',
 'co',
 'nvive',
 'con',
 'lo',
 'sobrenatural']

In [78]:
model = Word2Vec(sentences=oraciones_limpias, vector_size=500, window=5, min_count=1, workers=8)

In [81]:
palabras_cercanas = model.wv.most_similar('buendía', topn=10)
palabras_cercanas

[('segundo', 0.997196614742279),
 ('coronel', 0.9967167377471924),
 ('josé', 0.9946346879005432),
 ('promovió', 0.9855259656906128),
 ('raquel', 0.9844239354133606),
 ('confió', 0.9836245179176331),
 ('aureliano', 0.983380138874054),
 ('fusilarán', 0.9823739528656006),
 ('buendia', 0.982210636138916),
 ('seminario', 0.9812296032905579)]

In [82]:
model.wv.save_word2vec_format("100as_emb.txt", binary=False)

In [83]:
embeddings_cargados = KeyedVectors.load_word2vec_format("100as_emb.txt", binary=False)
def analogias(v1,v2,v3):
    similitud = embeddings_cargados.most_similar(positive=[v1,v3], negative=[v2])
    print(f"{v1} es a {v2} como {similitud[0][0]} es a {v3}")

In [84]:
analogias("caballero","hombre","mujer")

caballero es a hombre como llegó es a mujer


### Prueba modelo con muchos pdfs

In [ ]:
!pip install tqdm -q

In [115]:
import os
import PyPDF2
from tqdm import tqdm

In [116]:
def extraer_texto_desde_pdf(ruta_archivo):
    with open(ruta_archivo, "rb") as archivo:
        lector = PyPDF2.PdfReader(archivo)
        texto = ""
        for pagina in range(len(lector.pages)):
            texto += lector.pages[pagina].extract_text()
    return texto

In [117]:
ruta_carpeta = "./varios _pdf/"
todos_los_textos = []

for archivo in tqdm(os.listdir(ruta_carpeta)):
    if archivo.endswith(".pdf"):
        ruta_completa = os.path.join(ruta_carpeta, archivo)
        try:
            documento = extraer_texto_desde_pdf(ruta_completa)
            todos_los_textos.append(documento)
        except Exception as e:
            print(f"Error al procesar {archivo}: {e}")

 25%|██▌       | 34/134 [03:53<03:20,  2.00s/it]

Error al procesar 01. Introduccion a la Biología Evoluiva autor Marco A. Méndez y José Navarro B..pdf: PyCryptodome is required for AES algorithm
Error al procesar 01. Introducción a la automatización Autor Teacherke.pdf: PyCryptodome is required for AES algorithm


100%|██████████| 134/134 [15:44<00:00,  7.05s/it]


In [118]:
todos_los_textos[0]

'Alfonso Nieves Gómez©UNIVERSIDAD LIBRE, SEDE CARTAGENA \nAPUNTES DE DErEcho r om ANo \nAutor: Alfonso Nieves Gómez\nISBN: 978-958-8621-49-4  \nPrimera Edición, 2010 América del Sur \nTeléfonos: 661147- 6561379\nEditorial Universidad Libre, Sede Cartagena\ncomiTé EDiT oriAl\nRafael Ballestas Morales\nCarlos Gustavo Méndez Rodríguez\nArmando noriega Ruiz\nMaría Cristina Bustillo\nZilath Romero González\nDiagramación e Impresión: \nAlPhA  EDiT orES\nCartagena de Indias, Colombia Año 2014\nSe permite la reproducción total y parcial por cualquier \nmedio siempre y cuando se citen debidamente la \nfuente, los autores y las instituciones. La Universidad \nLibre, Sede Cartagena no se hace responsable por \nlos contenidos, posibles errores u omisiones. Los \ncontenidos son responsabilidad exclusiva de sus \nautores.UNiVErSiDAD liBrE \nDirEc TiVoS NA cioNAlES 2014 \nPresidente Nacional\nVíctor Hernando Alvarado Ardila\nrector Nacional\nNicolás Enrique Zuleta Hincapié\ncensor Nacional\nBenjamín 

In [119]:
oraciones_totales = []
tamaño = 0

for documento in todos_los_textos:
    tamaño = tamaño + len(documento)
    oraciones = documento.split(".")
    oraciones_totales.extend(oraciones)

In [120]:
print(f"Número de carácteres totales: {tamaño}")

Número de carácteres totales: 76976392


In [121]:
print(len(oraciones_totales))

1112072


In [122]:
oraciones_limpias = []
for oracion in oraciones_totales:
    # Eliminar puntuación y dividir por espacios
    tokens = oracion.translate(str.maketrans("","",string.punctuation)).split()
    
    # Convertir a minúsculas 
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Añadir solo si hay tokens
    if tokens:
        oraciones_limpias.append(tokens)

In [123]:
oraciones_limpias[0]

['alfonso',
 'nieves',
 'libre',
 'sede',
 'cartagena',
 'apuntes',
 'de',
 'derecho',
 'r',
 'om',
 'ano',
 'autor',
 'alfonso',
 'nieves',
 'gómez',
 'isbn',
 'primera',
 'edición',
 'américa',
 'del',
 'sur',
 'teléfonos',
 'editorial',
 'universidad',
 'libre',
 'sede',
 'cartagena',
 'comité',
 'edit',
 'orial',
 'rafael',
 'ballestas',
 'morales',
 'carlos',
 'gustavo',
 'méndez',
 'rodríguez',
 'armando',
 'noriega',
 'ruiz',
 'maría',
 'cristina',
 'bustillo',
 'zilath',
 'romero',
 'gonzález',
 'diagramación',
 'e',
 'impresión',
 'alpha',
 'edit',
 'ores',
 'cartagena',
 'de',
 'indias',
 'colombia',
 'año',
 'se',
 'permite',
 'la',
 'reproducción',
 'total',
 'y',
 'parcial',
 'por',
 'cualquier',
 'medio',
 'siempre',
 'y',
 'cuando',
 'se',
 'citen',
 'debidamente',
 'la',
 'fuente',
 'los',
 'autores',
 'y',
 'las',
 'instituciones']

In [124]:
model = Word2Vec(sentences=oraciones_limpias, vector_size=500, window=5, min_count=1, workers=12)

In [125]:
palabras_cercanas = model.wv.most_similar("noche", topn=10)
palabras_cercanas

[('mañana', 0.9067478775978088),
 ('calle', 0.7260098457336426),
 ('nave', 0.7242533564567566),
 ('vela', 0.7215090394020081),
 ('tarde', 0.719170093536377),
 ('batalla', 0.7172651290893555),
 ('pelea', 0.7017942070960999),
 ('madrugada', 0.6982910633087158),
 ('oscuridad', 0.6973426342010498),
 ('cama', 0.6966083645820618)]

In [126]:
palabras_cercanas = model.wv.most_similar("amor", topn=10)
palabras_cercanas

[('odio', 0.8147440552711487),
 ('placer', 0.7937672734260559),
 ('dolor', 0.7887848615646362),
 ('afecto', 0.7607267498970032),
 ('hombre', 0.7562950253486633),
 ('sentimiento', 0.750781238079071),
 ('espíritu', 0.7465107440948486),
 ('corazón', 0.7341040968894958),
 ('desprecio', 0.7281666994094849),
 ('respeto', 0.7279638648033142)]

In [127]:
model.save("mod_13textos.model")

In [128]:
model.wv.save_word2vec_format("emb_13textos.txt", binary=False)

In [129]:
from gensim.models import KeyedVectors

embeddings_cargados = KeyedVectors.load_word2vec_format("emb_13textos.txt", binary=False)

In [130]:
def analogias(v1,v2,v3):
    similitud = embeddings_cargados.most_similar(positive=[v1,v3], negative=[v2])
    print(f"{v1} es a {v2} como {similitud[0][0]} es a {v3}")

In [131]:
analogias("cielo","aire","tierra")

cielo es a aire como luna es a tierra


In [132]:
analogias("universo","estrellas","libro")

universo es a estrellas como discurso es a libro


In [147]:
analogias("oso","mamífero","salmón")

oso es a mamífero como braud es a salmón
